In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [3]:
cie_10 = pd.read_excel("../data/external/CIE-10 - sin_puntos_y_X.xlsx")
regiones = (
    pd.read_excel("../data/external/Esquema_Registro-2023.xlsx", sheet_name=2, header=6)
    .dropna(how="all", axis=1)
    .iloc[:-1, :-1]
)
regiones["Código Comuna"] = regiones["Código Comuna"].astype(int)
regiones["Código Región"] = regiones["Código Región"].astype(int)

## 1. Análisis Sociodemografico a lo largo del tiempo

En este analisis se quieren responder las siguientes preguntas:

1. ¿Cuántas consultas hay por año?
2. ¿Cuáles son los diagnósticos más frecuentes por año?
3. ¿Cuál es el **rango etario** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnóstico?
4. ¿Cuál es el **género/sexo** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
5. ¿Cuál es la **previsión** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
6. ¿Cuál es el **servicio de salud** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?
7. ¿Cuál es la **comuna** de las personas que se atienden ambulatoriamente en el Hospital del
Tórax? ¿Cómo cambia este perfil por cada uno de los diagnósticos?

Para responder estas preguntas se utilizará la base de datos de consultas ambulatorias
del sistema HIS del Instituto Nacional del Tórax. La base de datos contiene consultas desde
el año 2016 al 2022. Esta base contiene los datos para saber la frecuencia por diagnóstico,
rango etario y género.

A pesar de lo anterior, la base de datos de consultas carece de los últimos 3 datos a analizar
(previsón, servicio de salud y comuna de residencia). Debido a esta razón, se utilizará la base
de TrackCare para obtener tales datos.

Además, se quiere responder cada una de estas preguntas para cada una de las especialidades de
las consultas, diagnósticos y por tipo de consulta.


In [4]:
df_track = pd.read_csv("../data/processed/datos_limpios_track.csv", sep=";", encoding="latin-1")
df_his = pd.read_csv(
    "../data/processed/datos_limpios_diagnosticos.csv", sep=";", encoding="latin-1"
)

In [5]:
df_track["hora_completa_cita"] = pd.to_datetime(df_track["hora_completa_cita"])
df_his["fecha_reserva"] = pd.to_datetime(df_his["fecha_reserva"])

In [6]:
print(
    f"La base de Track tiene citas agendadas desde {df_track['hora_completa_cita'].min()} "
    f"a {df_track['hora_completa_cita'].max()}"
)

print(
    f"La base de HIS tiene citas agendadas desde {df_his['fecha_reserva'].min()} a "
    f"{df_his['fecha_reserva'].max()}"
)
print()
print(f"La base de Track tiene {df_track.shape[0]} citas")
print(f"La base de HIS tiene {df_his.shape[0]} citas")

La base de Track tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 20:30:00
La base de HIS tiene citas agendadas desde 2016-01-04 08:00:00 a 2022-12-30 14:50:00

La base de Track tiene 758350 citas
La base de HIS tiene 265441 citas


In [7]:
datos_pacientes_unicos_por_sesion = df_track.groupby(["id_paciente", "hora_completa_cita"]).head(1)

In [8]:
df_his_y_track = pd.merge(
    df_his,
    datos_pacientes_unicos_por_sesion,
    how="left",
    left_on=["id_paciente", "fecha_reserva"],
    right_on=["id_paciente", "hora_completa_cita"],
)

sin_datos_en_track = df_his_y_track[df_his_y_track.estadocita.isna()]

In [9]:
print(
    f"Luego de unir la base de HIS con Track, la primera ha quedado con "
    f"{df_his_y_track.shape[0]} consultas (de {df_his.shape[0]}). "
    f"Hay {sin_datos_en_track.shape[0]} consultas de HIS sin información en Track"
)

Luego de unir la base de HIS con Track, la primera ha quedado con 265441 consultas (de 265441). Hay 189 consultas de HIS sin información en Track


In [10]:
for paciente in sin_datos_en_track.id_paciente.unique():
    if paciente in df_track["id_paciente"]:
        print(f"{paciente} si esta")

Los resultados anteriores indican que ninguno de los pacientes faltantes se encuentra en la base
de Track (Y no ocurre debido a que los pacientes existen en la base, pero tienen una fecha de atencion distinta a la de HIS). 
Debido a lo anterior, estos pacientes carecen de datos sociodemográficos.

Una vez realizado la unión de las bases de datos es posible realizar el conteo anidado por
las variables sociodemográficas.

In [11]:
VARIABLES_SOCIODEMOGRAFICAS_INTERES = [
    "ano",
    "sexo_x",
    "comuna",
    "provincia",
    "region",
    "plan",
    "prevision",
    "tipoatencion",
    "rango_etario",
]

VARIABLE_A_CONTAR = ["nombre_diagnostico", "codigo_diagnostico"]

In [12]:
resultado_sociodemografico = (
    df_his_y_track.groupby(VARIABLES_SOCIODEMOGRAFICAS_INTERES)[VARIABLE_A_CONTAR]
    .value_counts()
    .reset_index(name="cantidad_de_consultas")
)

resultado_sociodemografico["comuna"] = resultado_sociodemografico["comuna"].str.upper()
resultado_sociodemografico = pd.merge(
    resultado_sociodemografico, regiones, how="left", left_on="comuna", right_on="Nombre Comuna"
).merge(cie_10, how="left", left_on="codigo_diagnostico", right_on="Código")

resultado_sociodemografico = resultado_sociodemografico.rename(
    columns={"Código Región": "codregion", "Código Comuna": "cod_comuna"}
)

resultado_sociodemografico["codregion"] = resultado_sociodemografico["codregion"].astype("Int32")
resultado_sociodemografico["cod_comuna"] = resultado_sociodemografico["cod_comuna"].astype("Int32")

In [13]:
resultado_sociodemografico.to_csv(
    "../data/interim/sociodemografico/desglose_sociodemografico.csv",
    index=False,
    encoding="latin-1",
    sep=";",
)

# Obtención de casos de Fibrosis Quística

En el Proyecto EPH del INT existe interés por saber la cantidad de consultas ambulatorias para la fibrosis quística. Por lo tanto, es necesario responder tal pregunta.

En primer lugar, hay que identificar todos los códigos diagnósticos que correspondan a la fibrosis quística.

In [14]:
# Genera una mask con las consultas que tengan 'fibrosis qu'. El nombre es pasado a minuscula
contienen_fibrosis_quistica = (
    resultado_sociodemografico["nombre_diagnostico"].str.lower().str.contains("fibrosis qu")
)

# Obtiene los codigos correspondientes a la fibrosis
resumen_fibrosis = resultado_sociodemografico[contienen_fibrosis_quistica]
codigos_fibrosis = resumen_fibrosis["codigo_diagnostico"].unique()
nombres_fibrosis = resumen_fibrosis["nombre_diagnostico"].unique()

print(
    f"Los códigos utiliados para la fibrosis quística son: {codigos_fibrosis} - {nombres_fibrosis}"
)

# Obtiene tabla dinámica con resultados de fibrosis
tabla_resumen_consultas_fibrosis = pd.pivot_table(
    resumen_fibrosis,
    index="rango_etario",
    columns="ano",
    values="cantidad_de_consultas",
    aggfunc="sum",
    fill_value=0,
    margins=True,
)

Los códigos utiliados para la fibrosis quística son: ['E840' 'E849' 'E841' 'E848' 'E84X' '23' '80'] - ['Fibrosis quística con manifestaciones pulmonares'
 'Fibrosis quística, sin otra especificación '
 'Fibrosis quística con manifestaciones intestinales'
 'Fibrosis quística con otras manifestaciones' 'Fibrosis quística '
 'Fibrosis quística infectada' 'Fibrosis Quistica sospecha']


In [15]:
tabla_resumen_consultas_fibrosis.to_excel("../data/interim/resumen_fibrosis_quistica_ambulatorio.xlsx")

In [16]:
# Define diagnosticos para los problemas de salud de interes
PROBLEMAS_DE_SALUD = {
    "FIBROSIS": ["E840", "E841", "E849", "E848"],
    "HIPERTENSION_PULMONAR": ["I270", "I272"],
    "FIBROSIS_PULMONAR": ["J841"],
    "ESCLERODERMIA": ["M348"],
    "MARFAN": ["Q874"],
    "EXCAVATUM": ["Q676"],
    "CARINATUM": ["Q677"],
    "SILICOSIS": ["J678", "J679"],
    "CARDIOPATIA_CONGENITA_ADULTO": [],
    "TRASPLANTE_DE_PULMON": [],
    "ECMO": [],
    "CANCER_DE_PULMON": [
        "C33X",
        "C340",
        "C341",
        "C342",
        "C343",
        "C381",
        "C384",
        "C450",
        "C780",
        "C782",
        "D381",
    ],
    "CIRUGIA_CANCER_DE_PULMON": [],
    "TRASPLNATE_DE_CORAZON": [],
    "ENFERMEDAD_CORONARIA": [],
    "ANEURISMA_DE_LA_AORTA": ["I712"],
    "DISECCION_DE_LA_AORTA": ["I710"],
    "MARCAPASOS": ["Z450"],
    "INSUFICIENCIA_MITRAL": ["I340"],
    "INSUFICIENCIA_AORTICA": ["I351"],
    "COVID": ["U071"],
    "APNEA_DEL_SUENO": ["G473"],
}

In [17]:
df_2019 = df_his_y_track.query("fecha_atencion < '2020-01-01'")

In [20]:
resultados = {}
for problema_de_salud, codigos_diagnostico in PROBLEMAS_DE_SALUD.items():
    # Filtra el df del torax solamente a los diagnosticos del problema
    df_diagnostico = df_2019[df_2019["codigo_diagnostico"].isin(codigos_diagnostico)]

    # Calcula metricas totales para el periodo analizado
    consultas_totales = df_diagnostico.shape[0]
    pacientes_totales = len(df_diagnostico["id_paciente"].unique())

    # Calcula metricas desagregadas por anios
    resumen_diagnostico_agrupado = (
        df_diagnostico.groupby(["ano"])
        .agg(n_egresos=("codigo_diagnostico", "count"), n_pacientes=("id_paciente", "nunique"))
        .mean()
    )
    # Selecciona la primera fila y la segunda columna del resumen ("n_egresos")
    consultas_promedios = resumen_diagnostico_agrupado["n_egresos"]
    pacientes_promedio = resumen_diagnostico_agrupado["n_pacientes"]

    # Genera el string resumen de metricas totales y promedio
    linea_resumen_consultas = f"{consultas_totales} ({consultas_promedios})"
    linea_resumen_pacientes = f"{pacientes_totales} ({pacientes_promedio})"

    # Guarda los strings resumen en un DataFrame
    resumen_problema_salud = pd.DataFrame(
        {
            "consultas_totales_2016_a_2019 (suma - promedio)": [linea_resumen_consultas],
            "personas_atendidas_2016_a_2019 (suma - promedio)": [linea_resumen_pacientes],
        },
        index=[problema_de_salud],
    )

    resultados[problema_de_salud] = resumen_problema_salud

In [23]:
resumen_total_diags = pd.concat(resultados.values())
display(resumen_total_diags)

,consultas_totales_2016_a_2019 (suma - promedio),personas_atendidas_2016_a_2019 (suma - promedio)
FIBROSIS,1999 (499.75),135 (87.0)
HIPERTENSION_PULMONAR,1703 (425.75),396 (215.25)
FIBROSIS_PULMONAR,7074 (1768.5),1613 (739.75)
ESCLERODERMIA,1 (1.0),1 (1.0)
MARFAN,28 (7.0),22 (5.75)
EXCAVATUM,248 (62.0),103 (42.5)
CARINATUM,41 (13.666666666666666),19 (8.333333333333334)
SILICOSIS,378 (94.5),160 (58.25)
CARDIOPATIA_CONGENITA_ADULTO,0 (nan),0 (nan)
TRASPLANTE_DE_PULMON,0 (nan),0 (nan)
